In [9]:
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import LLMResult

class MyCustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token:str, **kwargs) -> None:
        print(f"Received streaming token {token}, kwargs:{kwargs}")
        
    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"Received non-streaming response: {response}")

# Open AI

In [1]:
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import LLMResult

class MyCustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token:str, **kwargs) -> None:
        print(f"Received streaming token {token}, kwargs:{kwargs}")
        
    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"Received non-streaming response: {response}")

In [2]:
from llmstudio.llm.langchain import ChatLLMstudio
from llmstudio import LLM

In [3]:
# lc_llm = ChatLLMstudio(model_id='openai/gpt-3.5-turbo', temperature=0)
lc_llm = ChatLLMstudio(model_id='openai/gpt-3.5-turbo', temperature=0)

Running LLMstudio Engine on http://localhost:50001 Running LLMstudio Tracking on http://localhost:50002 

openai.py - request: api_key=None model='gpt-3.5-turbo' chat_input=[{'role': 'system', 'content': 'You are a helpful AI assistant.'}, {'role': 'user', 'content': 'When does my train depart? My ticket is 1234'}] parameters=OpenAIParameters(temperature=0.0, max_tokens=None, top_p=None, frequency_penalty=None, presence_penalty=None) is_stream=False has_end_token=False functions=[{'name': 'get_departure', 'description': 'get_departure(ticket_number: str) - Use this to fetch the departure time of a train', 'parameters': {'type': 'object', 'properties': {'ticket_number': {'type': 'string'}}, 'required': ['ticket_number']}}, {'name': 'cancel_ticket', 'description': 'cancel_ticket(ticket_number: str) - Use this to cancel a ticket', 'parameters': {'type': 'object', 'properties': {'ticket_number': {'type': 'string'}}, 'required': ['ticket_number']}}, {'name': 'buy_ticket', 'description': 'bu

In [6]:
lc_llm.invoke('Hello')

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b8a79a01-1f7f-4aca-9fe1-57753cfb85b2-0')

In [4]:
from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent

@tool
def get_departure(ticket_number: str):
    """Use this to fetch the departure time of a train"""
    return "12:00 AM"

@tool
def cancel_ticket(ticket_number: str):
    """Use this to cancel a ticket"""
    return "Ticket cancelled"

@tool
def buy_ticket(destination: str):
    """Use this to buy a ticket"""
    return "Bought ticket number 123456"

@tool
def make_complaint(complaint: str):
    """Use this to forward a complaint to the complaint department"""
    return "Complaint forwarded. We appreciate your feedback."


def assistant(question: str)->str:
    """YOUR CODE HERE"""
    tools = [get_departure, cancel_ticket, buy_ticket, make_complaint]
    print(tools)

    #rebuild agent with new tools
    agent_executor = initialize_agent(
        tools, lc_llm, agent=AgentType.OPENAI_FUNCTIONS, handle_parsing_errors="Check your output and make sure it conforms!", verbose = True, debug = True
    )

    response = agent_executor.invoke(
        {
            "input": question
        }
    )

    return response

In [5]:
assistant('When does my train depart? My ticket is 1234')


[StructuredTool(name='get_departure', description='get_departure(ticket_number: str) - Use this to fetch the departure time of a train', args_schema=<class 'pydantic.v1.main.get_departureSchema'>, func=<function get_departure at 0x13fe33b00>), StructuredTool(name='cancel_ticket', description='cancel_ticket(ticket_number: str) - Use this to cancel a ticket', args_schema=<class 'pydantic.v1.main.cancel_ticketSchema'>, func=<function cancel_ticket at 0x13fe32f20>), StructuredTool(name='buy_ticket', description='buy_ticket(destination: str) - Use this to buy a ticket', args_schema=<class 'pydantic.v1.main.buy_ticketSchema'>, func=<function buy_ticket at 0x149678360>), StructuredTool(name='make_complaint', description='make_complaint(complaint: str) - Use this to forward a complaint to the complaint department', args_schema=<class 'pydantic.v1.main.make_complaintSchema'>, func=<function make_complaint at 0x149678c20>)]


> Entering new AgentExecutor chain...


/opt/anaconda3/envs/llmstudiodev/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(



Invoking: `get_departure` with `{'ticket_number': '1234'}`


12:00 AMYour train departs at 12:00 AM.

> Finished chain.


{'input': 'When does my train depart? My ticket is 1234',
 'output': 'Your train departs at 12:00 AM.'}

In [ ]:
assistant('Hello')

# Vertex AI

In [1]:
from llmstudio.llm.langchain import ChatLLMstudio
from llmstudio import LLM

In [2]:
# lc_llm = ChatLLMstudio(model_id='openai/gpt-3.5-turbo', temperature=0)
lc_llm = ChatLLMstudio(model_id='vertexai/gemini-1.5-flash', temperature=0)

Running LLMstudio Engine on http://localhost:51816 Running LLMstudio Tracking on http://localhost:51817 



In [3]:
lc_llm.invoke('Hello')

AIMessage(content='Please provide me with the tool responses you received. I need that information to understand the context and answer your question.  \n', response_metadata={'token_usage': None, 'model_name': 'gemini-1.5-flash', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e33b97f8-7fca-47f7-ba60-0f629961d3ea-0')

In [3]:
from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent

@tool
def get_departure(ticket_number: str):
    """Use this to fetch the departure time of a train"""
    return "12:00 AM"

@tool
def cancel_ticket(ticket_number: str):
    """Use this to cancel a ticket"""
    return "Ticket cancelled"

@tool
def buy_ticket(destination: str):
    """Use this to buy a ticket"""
    return "Bought ticket number 123456"

@tool
def make_complaint(complaint: str):
    """Use this to forward a complaint to the complaint department"""
    return "Complaint forwarded. We appreciate your feedback."


In [4]:
def assistant(question: str)->str:
    """YOUR CODE HERE"""
    tools = [get_departure, cancel_ticket, buy_ticket, make_complaint]
    print(tools)

    #rebuild agent with new tools
    agent_executor = initialize_agent(
        tools, lc_llm, agent=AgentType.OPENAI_FUNCTIONS, handle_parsing_errors="Check your output and make sure it conforms!", verbose = True, debug = True
    )

    response = agent_executor.invoke(
        {
            "input": question
        }
    )

    return response

In [5]:
assistant('Can you buy me a ticket for madrid, and tell me when the train depparts?')

[StructuredTool(name='get_departure', description='get_departure(ticket_number: str) - Use this to fetch the departure time of a train', args_schema=<class 'pydantic.v1.main.get_departureSchema'>, func=<function get_departure at 0x136cc7060>), StructuredTool(name='cancel_ticket', description='cancel_ticket(ticket_number: str) - Use this to cancel a ticket', args_schema=<class 'pydantic.v1.main.cancel_ticketSchema'>, func=<function cancel_ticket at 0x136d3cb80>), StructuredTool(name='buy_ticket', description='buy_ticket(destination: str) - Use this to buy a ticket', args_schema=<class 'pydantic.v1.main.buy_ticketSchema'>, func=<function buy_ticket at 0x141d97740>), StructuredTool(name='make_complaint', description='make_complaint(complaint: str) - Use this to forward a complaint to the complaint department', args_schema=<class 'pydantic.v1.main.make_complaintSchema'>, func=<function make_complaint at 0x141d97f60>)]


> Entering new AgentExecutor chain...


/opt/anaconda3/envs/llmstudiodev/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(



Invoking: `buy_ticket` with `{'destination': 'madrid'}`


Bought ticket number 123456
Invoking: `get_departure` with `{'ticket_number': '123456'}`


12:00 AMI have bought a ticket for Madrid with ticket number 123456. The train departs at 12:00 AM. 


> Finished chain.


{'input': 'Can you buy me a ticket for madrid, and tell me when the train depparts?',
 'output': 'I have bought a ticket for Madrid with ticket number 123456. The train departs at 12:00 AM. \n'}

In [6]:
from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent

@tool
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True

@tool
def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."

@tool
def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True


In [7]:
def assistant(question: str)->str:
    """YOUR CODE HERE"""
    tools = [power_disco_ball, start_music, dim_lights]
    print(tools)

    #rebuild agent with new tools
    agent_executor = initialize_agent(
        tools, lc_llm, agent=AgentType.OPENAI_FUNCTIONS, handle_parsing_errors="Check your output and make sure it conforms!", verbose = True, debug = True
    )

    response = agent_executor.invoke(
        {
            "input": question
        }
    )

    return response

In [8]:
assistant('Turn this into a party!')

[StructuredTool(name='power_disco_ball', description='power_disco_ball(power: bool) -> bool - Powers the spinning disco ball.', args_schema=<class 'pydantic.v1.main.power_disco_ballSchema'>, func=<function power_disco_ball at 0x11d619760>), StructuredTool(name='start_music', description='start_music(energetic: bool, loud: bool, bpm: int) -> str - Play some music matching the specified parameters.\n\n    Args:\n      energetic: Whether the music is energetic or not.\n      loud: Whether the music is loud or not.\n      bpm: The beats per minute of the music.\n\n    Returns: The name of the song being played.', args_schema=<class 'pydantic.v1.main.start_musicSchema'>, func=<function start_music at 0x11d618fe0>), StructuredTool(name='dim_lights', description='dim_lights(brightness: float) -> bool - Dim the lights.\n\n    Args:\n      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.', args_schema=<class 'pydantic.v1.main.dim_lightsSchema'>, func=<function dim_lights at 0x

{'input': 'Turn this into a party!',
 'output': 'The disco ball is powered, the music is playing, and the lights are dimmed. This sounds like a party! \n'}

---

# Tests

### VertexAI request Format

In [ ]:
import pathlib
import textwrap
import time

import google.generativeai as genai

from IPython import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

#### Disco example

In [ ]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True

tools = [power_disco_ball, start_music, dim_lights]

In [ ]:
# Call the API.
model = genai.GenerativeModel('gemini-1.5-flash', tools=tools)
response = model.generate_content("Turn this into a party!")

print(response)

#### Train example

In [ ]:
def get_departure(ticket_number: str):
    """Use this to fetch the departure time of a train"""
    return "12:00 AM"

def cancel_ticket(ticket_number: str):
    """Use this to cancel a ticket"""
    return "Ticket cancelled"

def buy_ticket(destination: str):
    """Use this to buy a ticket"""
    return "Bought ticket number 123456"

def make_complaint(complaint: str):
    """Use this to forward a complaint to the complaint department"""
    return "Complaint forwarded. We appreciate your feedback."

tools = [get_departure, cancel_ticket, buy_ticket, make_complaint]

In [ ]:
# Call the API.
model = genai.GenerativeModel('gemini-1.5-flash', tools=tools)
response = model.generate_content("Can you buy me a ticket to madrid and tell me at what time the train depparts?")

print(response)

### Solution attempt

In [ ]:
import pathlib
import textwrap
import time

import google.generativeai as genai

from IPython import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
def get_departure(ticket_number: str):
    """Use this to fetch the departure time of a train"""
    return "12:00 AM"

def cancel_ticket(ticket_number: str):
    """Use this to cancel a ticket"""
    return "Ticket cancelled"

def buy_ticket(destination: str):
    """Use this to buy a ticket"""
    return "Bought ticket number 123456"

def make_complaint(complaint: str):
    """Use this to forward a complaint to the complaint department"""
    return "Complaint forwarded. We appreciate your feedback."

tools = [get_departure, cancel_ticket, buy_ticket, make_complaint]

In [ ]:
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest", tools=tools)

# Call the API.
chat = model.start_chat()
response = chat.send_message("Can you buy me a ticket for madrid and tell me at what time the train departs?")

# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")
     